In [15]:
#!pip3 install email-validate
# coding: utf8

In [21]:
import sys
#from imp import reload
#reload(sys)
print(sys.getdefaultencoding())

utf-8


In [22]:
import pandas as pd
import pymssql as psq
import psycopg2.extras
from email_validate import validate, validate_or_fail
import email_validate.exceptions as eex


Соединяемся с базой стек

In [30]:
mes_conn = psq.connect(server='192.168.9.10', user='LUBCHANSKIY', password='789456', database='Magadan_stack', charset='WINDOWS-1251')

In [45]:
def email_fizlic(s_conn):
    cursor = s_conn.cursor()
    sql_srting = u"SELECT лс.Номер лицевой, св.Примечание email FROM stack.Свойства св join stack.[Лицевые счета] лс ON лс.ROW_ID = св.[Счет-Параметры] where [Виды-Параметры]=177"
    sql_srting = sql_srting.encode('utf8')
    print(sql_srting)
    cursor.execute(sql_srting)
    row = cursor.fetchone()
    data = []
    while row:
        data.append(row)
        row = cursor.fetchone()      
    return data

In [32]:
def email_ulorg(s_conn):
    cursor = s_conn.cursor()
    cursor.execute("SELECT Название, орг.email FROM stack.[Организации] орг where email is not null and email != ''")
    row = cursor.fetchone()
    data = []
    while row:
        data.append(row)
        row = cursor.fetchone()      
    return data

In [46]:
fiz_email_list = email_fizlic(mes_conn)
ul_email_list = email_ulorg(mes_conn)

b'SELECT \xd0\xbb\xd1\x81.\xd0\x9d\xd0\xbe\xd0\xbc\xd0\xb5\xd1\x80 \xd0\xbb\xd0\xb8\xd1\x86\xd0\xb5\xd0\xb2\xd0\xbe\xd0\xb9, \xd1\x81\xd0\xb2.\xd0\x9f\xd1\x80\xd0\xb8\xd0\xbc\xd0\xb5\xd1\x87\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5 email FROM stack.\xd0\xa1\xd0\xb2\xd0\xbe\xd0\xb9\xd1\x81\xd1\x82\xd0\xb2\xd0\xb0 \xd1\x81\xd0\xb2 join stack.[\xd0\x9b\xd0\xb8\xd1\x86\xd0\xb5\xd0\xb2\xd1\x8b\xd0\xb5 \xd1\x81\xd1\x87\xd0\xb5\xd1\x82\xd0\xb0] \xd0\xbb\xd1\x81 ON \xd0\xbb\xd1\x81.ROW_ID = \xd1\x81\xd0\xb2.[\xd0\xa1\xd1\x87\xd0\xb5\xd1\x82-\xd0\x9f\xd0\xb0\xd1\x80\xd0\xb0\xd0\xbc\xd0\xb5\xd1\x82\xd1\x80\xd1\x8b] where [\xd0\x92\xd0\xb8\xd0\xb4\xd1\x8b-\xd0\x9f\xd0\xb0\xd1\x80\xd0\xb0\xd0\xbc\xd0\xb5\xd1\x82\xd1\x80\xd1\x8b]=177'


ProgrammingError: (102, b"Incorrect syntax near '\xbb'.DB-Lib error message 20018, severity 15:\nGeneral SQL Server error: Check messages from the SQL Server\n")

In [7]:
def valid_email(email):
    try:
        validate_or_fail(
            email_address=email,
            check_format=True,
            check_blacklist=False,
            check_dns=True,
            dns_timeout=10,
            check_smtp=False,
            smtp_debug=False
        )
    except eex.DomainNotFoundError:
        return 'Адрес домена не найден'
    except eex.NoNameserverError:
        return ''
    except eex.DNSTimeoutError:
        return 'Истекло время ожидания при запросе сервера имен.'
    except eex.DNSConfigurationError:
        return 'Сервер имен настроен неправильно.'
    except eex.NoMXError:
        return 'Сервер имен не содержит записей MX для домена.'
    except eex.NoValidMXError:
        return 'Сервер имен перечисляет записи MX для домена, но ни одна из них не является допустимой.'
    except eex.AddressFormatError:
        return 'Формат адреса не соответствует требованиям'
    except eex.DomainBlacklistedError:
        return 'Адрес домена в черном списке!'
    else:
        return ''

In [8]:
email_errors = []
for email_ls in fiz_email_list:
    res = valid_email(email_ls[1])
    if res != '':
        email_errors.append([email_ls[0], email_ls[1], res])
email_df = pd.DataFrame(email_errors)
email_df

,0,1,2
0,111408,muchkaeva@mdnab.su,Адрес домена не найден
1,120238,fred_magadan13@maiI.ru,Сервер имен не содержит записей MX для домена.
2,130375,tam_mazur.@mail.ru,Формат адреса не соответствует требованиям
3,222196,olya-olechka-2012@maiI.ru,Сервер имен не содержит записей MX для домена.
4,230136,ivachepele@gmail.ru,Сервер имен не содержит записей MX для домена.
5,240628,naiIevna2015@maiI.ru,Сервер имен не содержит записей MX для домена.
6,241867,inna.aIzhanova@maiI.ru,Сервер имен не содержит записей MX для домена.
7,320349,bev28ym@maiI.ru,Сервер имен не содержит записей MX для домена.
8,321743,jv.turulina@maik.ru,Сервер имен не содержит записей MX для домена.
9,340739,polyarnya@gmali.com,Сервер имен не содержит записей MX для домена.


In [12]:
ul_email_errors = []
for email_ls in ul_email_list:
    if ';' in email_ls[1]:
        spl_res = email_ls[1].split(';')
        for eml in spl_res:
            res = valid_email(eml)
            if res != '':
                ul_email_errors.append([email_ls[0], eml, res])
    elif  ',' in email_ls[1]:
        spl_res = email_ls[1].split(',')
        for eml in spl_res:
            res = valid_email(eml)
            if res != '':
                ul_email_errors.append([email_ls[0], eml, res])
    else:
        res = valid_email(email_ls[1])
        if res != '':
            ul_email_errors.append([email_ls[0], email_ls[1], res])
            
ulemail_df = pd.DataFrame(ul_email_errors)
ulemail_df

,0,1,2
0,"ÎÎÎ ""Ëþáèìûé ãîðîä""",info@ooouklg.ru,Адрес домена не найден
1,ÒÅÑÒÎÂÛÉ ÏËÀÒÅËÜÙÈÊ ÞË ¹1,test@test.ru,Адрес домена не найден
2,"ÀÎ ""ÊÎËÛÌÑÊÀß ÐÎÑÑÛÏÜ""",buhtmc@kolymros.ru,Адрес домена не найден
3,"ÀÎ ""ÎÀÂÑ""",alexgv777@mail.ru,Формат адреса не соответствует требованиям
4,"ÀÎ ""ÎÀÂÑ""",kozlenko777@yandex.ru,Формат адреса не соответствует требованиям
5,"ÃÊÎÓ ""ÌÎÖÎ ¹ 1""",mogkou19@yandex.ru,Формат адреса не соответствует требованиям
6,"ÃÑÊ ""Ñîäðóæåñòâî""",zakup@meser.su,Адрес домена не найден
7,ÄÀËÜÍÅÄÐÀ,dalnedra@rosnedrs.gov.ru,Адрес домена не найден
8,"ÌÀÓ ""ÑÊ""ÌÅÒÀËËÈÑÒ""",cport.metallict49@mail.ru,Формат адреса не соответствует требованиям
9,"ÌÀÓÊ ã. Ìàãàäàíà ""Êèíîòåàòð""Ãîðíÿê""/",info@kinomagadan.ru,Формат адреса не соответствует требованиям


In [28]:
def enc_str(str_):
    return str_.decode('utf8').encode('windows-1251')

In [29]:
ulemail_df['enc_1'] = ulemail_df[0].apply(enc_str)
ulemail_df

AttributeError: 'str' object has no attribute 'decode'

In [41]:
# Запихиваем все в Excel
sh_name = 'Ошибки'
writer = pd.ExcelWriter('email_error.xlsx', engine='xlsxwriter')

email_df.to_excel(writer, sheet_name=sh_name, index=False)

workbook = writer.book
worksheet = writer.sheets[sh_name]
writer.save()
writer.close()


c:\Users\Lubchanskiy\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [29]:
print(len(fiz_email_list))

2213


In [27]:
ulemail_df.to_csv('ul_error.csv')